In [1]:
"""
1. Paste token contract
2. Look at top 100 holders
    - holders balance
        -> 100k+
        -> 50 - 100k
        -> 10 - 50k
        -> 1 - 10k
        -> 0 - 1k 
    
    - holders transaction numbers
        fresh wallet - less than 10 transactions
        good wallets - in between
        jeet - more than ____ transactions

    - supply distribution between such holders and contracts

    ignore 0x000000000000000000000000000000000000dead address

3. 
        

"""

import requests
import pandas as pd
import json
import helper

pd.set_option('display.max_rows', None)

weiDivisible = 1000000000000000000.0
tokenContract = "0xc15ecfe8e11c6a1d8b51bd29e6866fea005bf943"
url = f"https://api.ethplorer.io/getTopTokenHolders/{tokenContract}?apiKey=freekey&limit=100"

response = requests.get(url)
response = response.json()
response = response['holders']
#data = json.loads(response)

df = pd.DataFrame.from_dict(response)

In [2]:
addresses = df['address']
isContract = []
for address in addresses:
    if(helper.isContract(address)):
        isContract.append(True)
    else: 
        isContract.append(False)
    
df["isContract"] = isContract
df2 = df[df["isContract"] == False]

In [3]:
addresses = df2["address"]
ethBalances = []
tokenBalances = []
for address in addresses:
    url = f"https://api.ethplorer.io/getAddressInfo/{address}?apiKey=EK-jCkqf-qQHnUdG-uyNuL"

    response = requests.get(url)
    response = response.json()
    
    ethBalance = response['ETH']['balance'] * response['ETH']['price']['rate']
    tokenBalance = helper.getTokenBalance(address)

    ethBalances.append(ethBalance)
    tokenBalances.append(tokenBalance)

df.loc[:,'EthBalance'] = ethBalances
df.loc[:,'TokenBalance'] = tokenBalances
df.loc[:,'TotalBalance'] = df2["EthBalance"] + df2["TokenBalance"]

C:\Users\wwwbl\AppData\Local\Temp\ipykernel_1260\1123332923.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["EthBalance"] = ethBalances
C:\Users\wwwbl\AppData\Local\Temp\ipykernel_1260\1123332923.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["TokenBalance"] = tokenBalances
C:\Users\wwwbl\AppData\Local\Temp\ipykernel_1260\1123332923.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [4]:
#   RESEARCH FINDINGS
# Fresh: Total Transaction <= 10 
# Good:  Daily Transaction <= 5 (Inactive wallets also 'Good')
# Aight: Daily Transaction 5 < x <= 10
# Jett:   Daily Transaction > 10

wallets = df2["address"]
walletLabel = []

for wallet in wallets:
    url=f"https://api.etherscan.io/api?module=account&action=tokentx&address={wallet}&page=1&offset=100&startblock=0&endblock=99999999&sort=desc&apikey=7YRRHUFSDSM5BXCIM8FY3NPJ192WYKRC1G"

    resp=requests.get(url)
    resp=resp.json()
    resp=resp['result']

    if len(resp) < 10: 
        walletLabel.append("Fresh")
        continue

    yesterdayTime = int(resp[0]['timeStamp']) - 86400
    walletTransfers = []
    for transfer in resp:
        if int(transfer['timeStamp']) >= yesterdayTime: walletTransfers.append(transfer['timeStamp'])

    walletTransfers = [*set(walletTransfers)]
    dailyTransfer = len(walletTransfers)

    if dailyTransfer <= 5: walletLabel.append("Good")
    elif dailyTransfer > 5 and dailyTransfer <= 10: walletLabel.append("Aight")
    if dailyTransfer > 10: walletLabel.append("Jett")

df.loc[:,'Label'] = walletLabel

In [ ]:
df.to_csv("result.csv")